In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn_crfsuite import CRF
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


2023-05-11 10:25:31.157591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
!pip install sklearn-crfsuite
!pip install conllu

In [9]:
import io
from conllu import parse_incr

with io.open("en-train.conllu", "r", encoding="utf-8") as f:
    data = f.read()
    
#Split the data into sentences by splitting on the newline character
sentences = data.strip().split("\n")

In [10]:
array=[]
c=[]
for i in sentences:
    if i.startswith("# id"):
        array.append(c)
        c=[]
    else:
        c.append(i)
array = [subarray for subarray in array if subarray]

In [11]:
dt=[]
dl=[]
for i in array:
    #print(i)
    cc=[]
    tt=[]
    for j in i:
        if j!="":
            text=j.split(" _ _ ")
            cc.append(text[0])
            tt.append(text[1])
    dt.append(cc)
    dl.append(tt)

In [12]:
def word2features(sent, i):
    word = sent[i]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        prev_word = sent[i-1]
        features.update({
            'prev_word.lower()': prev_word.lower(),
            'prev_word.istitle()': prev_word.istitle(),
            'prev_word.isupper()': prev_word.isupper()
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        next_word = sent[i+1]
        features.update({
            'next_word.lower()': next_word.lower(),
            'next_word.istitle()': next_word.istitle(),
            'next_word.isupper()': next_word.isupper()
        })
    else:
        features['EOS'] = True
    return features

In [13]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return sent

In [14]:
X = [sent2features(s) for s in dt]
y = [sent2labels(s) for s in dl]


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn_crfsuite import metrics
crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=False)

# Fit the CRF model on the training data
crf.fit(X_train, y_train)

# Use the CRF model to make predictions on the testing data
y_pred = crf.predict(X_test)

# Print the classification report and F1 score
#print(metrics.flat_classification_report(y_test, y_pred))
print(f"F1 Score: {metrics.flat_f1_score(y_test, y_pred, average='weighted'):.2f}")


F1 Score: 0.84


In [3]:
import io
from conllu import parse_incr
with io.open("en-dev.conllu", "r", encoding="utf-8") as f:
    t_data = f.read()
    
#Split the data into sentences by splitting on the newline character
test_sentences = t_data.strip().split("\n")

In [6]:
t_array=[]
t_c=[]
for i in test_sentences:
    if i.startswith("# id"):
        t_array.append(t_c)
        t_c=[]
    else:
        t_c.append(i)
t_array = [subarray for subarray in t_array if subarray]

In [7]:
tdt=[]
tdl=[]
for i in t_array:
    #print(i)
    tcc=[]
    ttt=[]
    for j in i:
        if j!="":
            ttext=j.split(" _ _ ")
            tcc.append(ttext[0])
            ttt.append(ttext[1])
    tdt.append(tcc)
    tdl.append(ttt)

In [16]:
tX = [sent2features(s) for s in tdt]
ty = [sent2labels(s) for s in tdl]

In [18]:
_, tX_test, _, ty_test = train_test_split(tX, ty, test_size=0.2, random_state=42)

In [20]:
from sklearn_crfsuite import metrics
crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=False)

# Fit the CRF model on the training data
crf.fit(X_train, y_train)

# Use the CRF model to make predictions on the testing data
y_pred = crf.predict(tX_test)

# Print the classification report and F1 score
#print(metrics.flat_classification_report(y_test, y_pred))
print(f"F1 Score: {metrics.flat_f1_score(ty_test, y_pred, average='weighted'):.2f}")


F1 Score: 0.85
